<a href="https://colab.research.google.com/github/BrahianCarrera/KaggleCompetition/blob/main/03%20-%20preprocesado%20moda%20y%20RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
# Carga el archivo kaggle.json
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai4eng-20241
!unzip udea-ai4eng-20241.zip

Saving kaggle.json to kaggle.json
 61% 13.0M/21.2M [00:00<00:00, 62.1MB/s]
100% 21.2M/21.2M [00:00<00:00, 73.3MB/s]
Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')
df.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


# **Preprocesado**

EL preprocesado que defino por moda consta de en primera instancia limpiar la columna de programa, primero quitando las tildes y luego agrupando los datos que coincidan en su primera frase, además para los datos atipicos que la funcion de obtener frase no agruapa correctamente se les asigna su reemplazo manualmente a traves de un diccionario.

- Se eliminan las columnas de estu privado de la libertdad y la de fami tiene internet repetida, además de la periodo en la ultima ejecuccion que realice.
- Se renombran los indices para una manipulación mas comoda.
- Se convierten los valores boleanos a binarios
- Se imputan los datos categoricos por la moda de cada categoria
- Se transforma en one Hot todas las columnas
- Se transforma la columna de rendimiento Global a valores ordinales.

In [ ]:
def obtener_primera_frase(cadena):
    return cadena.split(' ', 1)[0]

def quitar_tildes(palabra):
    return re.sub(r'[ÁÉÍÓÚ]', lambda x: {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}[x.group()], palabra)

df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(obtener_primera_frase)
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(quitar_tildes)

reemplazos = {
    'ADMINISTRACION': ['ADMINISTRACION', 'ADMINISTRACI¿N', 'ADMINISTRACIÒN', 'ADMINISTRACIÓN', 'ADMINSITRACION', 'ADMINSITRACIÓN'],
    'ANTROPOLOGIA': ['ANTROPOLOGIA', 'ANTROPOLOGÍA'],
    'ARTES': ['ARTE', 'ARTES','BELLAS'],
    'BIOINGENIERIA': ['BIOINGENIERIA', 'BIOINGENIERÍA'],
    'BIOLOGIA': ['BIOLOGIA', 'BIOLOGÍA'],
    'COMUNICACION': ['COMUNICACION', 'COMUNICACIONES', 'COMUNICACI¿N', 'COMUNICACIÓN'],
    'CONSTRUCCION': ['CONSTRUCCION', 'CONSTRUCCIONES'],
    'CONTADURIA': ['CONTADURIA', 'CONTADURÍA'],
    'DISEÑO': ['DISENO', 'DISE¿O', 'DISE¿¿O', 'DISEÑO'],
    'ECOLOGIA': ['ECOLOGIA', 'ECOLOGÍA'],
    'ECONOMIA': ['ECONOM¿A'],
    'GEOLOGIA': ['GEOLOG¿A'],
    'GERENCIA': ['GERONTOLOGIA'],
    'INGENIERIA': ['INGENIER¿A', 'INGENIERÌA', 'INGENIIERIA'],
    'INSTRUMENTACION': ['INTRUMENTACION'],
    'MATEMATICAS': ['MATEMATICA'],
    'MARKETING': ['MERCADEO', 'MERCADOLOGIA'],
    'PSICOLOGIA': ['PSICOLOGA'],
    'QUIMICA': ['QUIMICA.', 'QU¿MICA'],
    'MUSICA': ['MUSICA-'],
    'TERAPIAS': ['TERAPIA', 'TERAPIAS'],
    'GESTION': ['GESTI¿N'],
    'NEGOCIOS' : ['3°'],
    'DEPORTE' : ['ACTIVIDAD'],
    'LICENCIATURA' : ['Licenciatura'],
    'LENGUAS' :['LENGUAJES']

}

for key, values in reemplazos.items():
    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].replace(values, key)



##Eliminar columnas

df = df.drop(labels =['ESTU_PRIVADO_LIBERTAD','FAMI_TIENEINTERNET.1','PERIODO'],axis= 1)


##Renombrar Columnas
##'PERIODO'
df.columns = ['ID','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE','RENDIMIENTO_GLOBAL']

##COnvertir a binario

def convertir_num(valor):
    if valor == 'Si':
        return 1
    elif valor == 'No':
        return 0
    else:
        return valor  # Si no es 'Si' ni 'No', dejar el valor original

# Aplicar la función a todas las columnas del DataFrame
df = df.applymap(convertir_num)

##Imputar Datos

df['ESTRATO'] = df['ESTRATO'].str.replace('Estrato ', '')

columnas_binarias = ['TIENE_INTERNET', 'TIENE_LAVADORA', 'TIENE_AUTO',
                      'PAGA_MATRICULA', 'TIENE_COMPUTADOR']

for columna in columnas_binarias:
    moda = df[columna].mode()[0]
    df[columna].fillna(moda, inplace=True)


ModaEstrato = df['ESTRATO'].mode()[0]  # Obtener la moda
df['ESTRATO'].fillna(ModaEstrato, inplace=True)

ModaPadre = df['EDUCACION_PADRE'].mode()[0]
df['EDUCACION_PADRE'].fillna(ModaPadre, inplace=True)

ModaMadre = df['EDUCACION_MADRE'].mode()[0]
df['EDUCACION_MADRE'].fillna(ModaMadre, inplace=True)

ValorMatricula = df['VALOR_MATRICULA'].mode()[0]
df['VALOR_MATRICULA'].fillna(ValorMatricula, inplace=True)

HorasTrabaja = df['HORAS_TRABAJA'].mode()[0]
df['HORAS_TRABAJA'].fillna(HorasTrabaja, inplace=True)

##Convertir variable de rendimiento global

df["RENDIMIENTO_GLOBAL"].value_counts()

valores_rendimiento = {'alto': 4,'medio-alto': 3,'medio-bajo':2, 'bajo': 1}
df= df.replace(valores_rendimiento)

##Convertir columnas categoricas
Columnas_categoricas = ["VALOR_MATRICULA","PROGRAMA","DEPARTAMENTO","EDUCACION_MADRE","EDUCACION_PADRE"]

df[Columnas_categoricas] = df[Columnas_categoricas].astype("category")

##Convertir a Onehot
encoder = OneHotEncoder(sparse=False)

encoded_columns = encoder.fit_transform(df[['HORAS_TRABAJA', 'VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE']])##'PERIODO'

feature_names = encoder.get_feature_names_out(['HORAS_TRABAJA', 'VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'])##'PERIODO'

one_hot_encoded_df = pd.DataFrame(encoded_columns, columns=feature_names)

df = pd.concat([df, one_hot_encoded_df], axis=1)

df = df.drop(labels =['HORAS_TRABAJA', 'VALOR_MATRICULA','PROGRAMA','DEPARTAMENTO','ESTRATO','EDUCACION_PADRE','EDUCACION_MADRE'],axis= 1)##'PERIODO'


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# **Separacion de los datos**

En esta seccion se separan los datos en una proporcion 70/30, ademas mientras hacia pruebas defini el random state para que siempre me otorgara los mismos datos para las pruebas.

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

print("Conjunto de entrenamiento:")
print(train_df)

print("\nConjunto de prueba:")
print(test_df)

# **Entrenamiento**

En esta seccion se entreno el modelo RandomForest con los datos previamente particionados, como se puede evidenciar no continue con el proceso de predecir los valores para el df de test puesto que cada ejecución del entrenamiento del modelo tardaba alrededor de 30 minutos y habian datos atipicos en el df de test que no estuvieron presentes en el entrenamiento. La presencia de estos datos atipicos implicaba reentrenar el modelo y desperdiciar aun más tiempo.

Además de lo previamente mencionado en el momento que entrenaba este modelo encontre otro que era mucho mas eficiente en los tiempos de entrenamiento y en los puntajes que arrojaba por lo que decisti de seguir entrenando a este modelo.


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


# Dividir los datos en características (X) y variable objetivo (y)
X = train_df.drop("RENDIMIENTO_GLOBAL", axis=1)  # Características
y = train_df["RENDIMIENTO_GLOBAL"]  # Variable objetivo


# Paso 2: Validación cruzada para la selección del modelo
rf = RandomForestClassifier()
scores = cross_val_score(rf, X, y, cv=5)  # Validación cruzada con 5 folds
print("Cross-validation scores:", scores)
print("Mean CV score:", scores.mean())

# Paso 3: Entrenamiento del modelo y generación de predicciones
rf.fit(X, y)

#preds = rf.predict(test_df)


Cross-validation scores: [0.38220732 0.38263022 0.38337287 0.38398143 0.38503352]
Mean CV score: 0.38344507478081485


RandomForestClassifier()